# Route Analysis Api
## Introduction



## Choose dates and other parameters here

In [2]:
from datetime import datetime
import requests
import json

import athena

import utils

date = "2018-09-23" # Change this and then execute the cell. Must use YYYY-MM-DD format.

# The apiHelper module defines northTollPlazaLoc, southTollPlazaLoc, northExitLoc, and southExitLoc.
datetime_object = datetime.strptime(date, "%Y-%m-%d")
day_of_the_week = utils.days[datetime_object.weekday()]
requestObject = {
  "jobName":"Test job",
  "distanceUnit":"MILES",
  "routes":[
    {
      "name":"NorthToSouth",
      "start":utils.northTollPlazaLoc,
      "via":[],
      "end":utils.southExitLoc,
      "fullTraversal":False,
      "zoneId":"US/Central",
      "probeSource":"ALL"
    },
    {
      "name":"SouthToNorth",
      "start":utils.southTollPlazaLoc,
      "via":[],
      "end":utils.northExitLoc,
      "fullTraversal":False,
      "zoneId":"US/Central",
      "probeSource":"ALL"
    }
  ],
    "dateRanges":[{
        "name":datetime_object.strftime("%a %b %-d %Y"), # Again this name is just for our convenience and can be anything.
        "from":date,
        "to":date
    }],
  "timeSets": utils.get_24_hours(day_of_the_week) # We want all 24 hour buckets.
}

print("Created request object for {}. Ready to POST a request.".format(date))

Created request object for 2018-09-23. Ready to POST a request.


## Execute this cell to send a new request to TomTom
This sends the request to TomTom. It will return a jobId. We will use that jobId to ask TomTom if it is done processing the request in the next cell.

NOTE: There seems to be some problem in verifying the SSL/TLS cert for api.tomtom.com, so I have disabled verification. Because this prints an ugly warning message I capture (hide) everything except stdout with %%capture.

In [4]:
%%capture --no-stdout

API_KEY = athena.tom_tom.get_api_key()

url = "https://api.tomtom.com/traffic/trafficstats/routeanalysis/1?key={}".format(API_KEY)
r = requests.post(url, verify=False, json=requestObject)
if (r.status_code == 200):
    print("Successful POST")
    job_id = json.loads(r.text)["jobId"]
    print("jobId = " + job_id)
else:
    print("Error in POST")
    print(r.status_code)
    print(r.text)

Successful POST
jobId = 529379


## Execute this cell to check on a request
This cell will make a call to the TomTom API to check on the status of the job `jobId`.

When the job is complete the response will contain urls to geojson, json, and zip files that contain the results. We exclusively use the geojson file. You can manually copy and paste the url into your browser and the file will download.

NOTE: There seems to be some problem in verifying the SSL/TLS cert for api.tomtom.com, so I have disabled verification. Because this prints an ugly warning message I capture (hide) everything except stdout with %%capture.

In [5]:
%%capture --no-stdout
url_get = "https://api.tomtom.com/traffic/trafficstats/status/1/{}?key={}".format(job_id, API_KEY)
response = requests.get(url_get, verify=False)
print(response.status_code)
print(response.text)

200
{"responseStatus":"OK","jobId":"529379","jobState":"DONE","urls":["https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/12/529379/a2f89b0f/Test_job.geojson.zip","https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/12/529379/a2f89b0f/Test_job.json","https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/12/529379/a2f89b0f/Test_job.kmz","https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/12/529379/a2f89b0f/Test_job.shapefile.zip","https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/12/529379/a2f89b0f/Test_job.xlsx"]}
